# Задание 1. (10 баллов)

Дообучите языковую модель на датасете инструкций, используя LoRA. Проверьте, что дообученная модель отличается от изначальной - сгенерируйте продолжения для одних и тех же промптов и сравните результаты.

Вы можете взять за основу код семинара PEFT, изменив датасет цитат на датасет инструкций (можно просто скопировать из семинара про General_instruct_fine-tuning).
Можно использовать alpaca_dataset, датасет Dolly 2 или переведенный датасет (или все вместе).
Важно использовать модель с большим количеством параметров (относительно семинара по General instruct fine-tuning).
Размер модели должен быть как минимум 3 млрд параметров.  
**Нужно использовать модель, которую мы не разбирали на семинаре (OPT-2.7b, OPT-6.7b). Найдите новую модель на huggingface hub.**



In [ ]:
# !pip install -q bitsandbytes datasets accelerate loralib
# !pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git
# так устанавливаются самые новые версии библиотек

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 17.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, BitsAndBytesConfig

In [ ]:
quantization_config = BitsAndBytesConfig(
        load_in_8bit=True
    )

In [ ]:
model = AutoModelForCausalLM.from_pretrained("Llama3-ChatQA-1.5-8B")
tokenizer = AutoTokenizer.from_pretrained("tokenizer-Llama3-ChatQA-1.5-8B")

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Модель

Попробуем загрузить большую модель без дополнительных параметров.

Размер модели должен быть как минимум 3 млрд параметров.
Нужно использовать модель, которую мы не разбирали на семинаре (OPT-2.7b, OPT-6.7b). Найдите новую модель на huggingface hub.

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "nvidia/Llama3-ChatQA-1.5-8B",
    quantization_config=quantization_config,
    cache_dir='./models'
)

config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/28.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/6.08G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("nvidia/Llama3-ChatQA-1.5-8B")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
model.save_pretrained("Llama3-ChatQA-1.5-8B")

In [ ]:
tokenizer.save_pretrained("tokenizer-Llama3-ChatQA-1.5-8B")

('tokenizer-Llama3-ChatQA-1.5-8B/tokenizer_config.json',
 'tokenizer-Llama3-ChatQA-1.5-8B/special_tokens_map.json',
 'tokenizer-Llama3-ChatQA-1.5-8B/tokenizer.json')

In [ ]:
model = AutoModelForCausalLM.from_pretrained("Llama3-ChatQA-1.5-8B")

In [ ]:
!ls

dataset.json  Llama3-ChatQA-1.5-8B  models  sample_data


## Датасет ru_turbo_alpaca

In [ ]:
#!pip uninstall torch torchvision torchaudio torchtext

In [ ]:
#!pip install torch==2.2.2 torchvision
#!pip install transformers[torch] pandas tokenizers datasets xformers zstandard jsonlines

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
import copy
import logging
from dataclasses import dataclass, field
from typing import Optional, Dict, Sequence
import json
import torch
import transformers
from torch.utils.data import Dataset
from transformers import Trainer

In [ ]:
dataset = load_dataset("IlyaGusev/ru_turbo_alpaca")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for IlyaGusev/ru_turbo_alpaca contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/IlyaGusev/ru_turbo_alpaca
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `d

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'alternative_output', 'label', 'all_labels', 'agreement', 'overlap'],
        num_rows: 29822
    })
})

In [ ]:
dataset['train'][:3]

{'instruction': ['Опишите, как сделать горшок из глины.',
  'Опиши процесс изготовления торта. Укажи ингредиенты, необходимые для этого.',
  'Напиши аналог данной пословицы на современный лад: "Не говори гоп, пока не перепрыгнешь".'],
 'input': ['', '', ''],
 'output': ['Для изготовления горшка из глины сначала необходимо взять глину и размешать ее водой до состояния пластилина. Затем нужно сформировать глиняный комок нужной формы и тщательно откачать влагу из глины, чтобы она стала твердой. Горшок нужно обжечь в духовке при высокой температуре, чтобы он стал прочным и непроницаемым.',
  'Для изготовления торта необходимы следующие ингредиенты: мука, сахар, яйца, молоко, сливочное масло, разрыхлитель, ванилин. Сначала нужно смешать муку с сахаром и добавить яйца, молоко, разрыхлитель и ваниль. Затем тесто надо хорошо перемешать и разделить на несколько частей. Каждую часть нужно запекать в духовке до готовности. После этого торт надо собрать, сложив слои пирога вместе с начинкой и укра

In [ ]:
dataset['train'][3]

{'instruction': 'Представьте, что вы занимаетесь дизайном главной страницы для нового сайта. Опишите, что вы хотели бы видеть на этой странице.',
 'input': '',
 'output': 'Я бы хотел, чтобы главная страница была простой, но информативной. На ней должны быть ясно выделены основные разделы и функции сайта, а также действительно привлекательный визуальный дизайн. Я бы использовал яркие цвета и простые шрифты, чтобы все было легко читаемо. Также я бы добавил крупные изображения и логотип компании, чтобы сделать страницу более запоминающейся.',
 'alternative_output': 'Как СПА, мной, ИИ, нет предпочтений и личных вкусов. Но как эксперт, я могу предложить следующее:\n\n1. Ясный и привлекательный заголовок, который бы кратко и точно передавал суть сайта. Например, "Лучший маркетплейс для покупки и продажи товаров".\n\n2. Навигационная панель, которая бы позволила посетителям легко перемещаться по сайту. Должно быть понятно, где можно найти продукты, услуги, информацию о компании, контакты, обр

In [ ]:
IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = "[PAD]"
DEFAULT_EOS_TOKEN = "</s>"
DEFAULT_BOS_TOKEN = "</s>"
DEFAULT_UNK_TOKEN = "</s>"

PROMPT_DICT = {
    "prompt_input": (
        "Ниже приведено описание задания, сопровождаемое входными данными, которые предоставляют дополнительный контекст. "
        "Напиши ответ, который соответствует запросу.\n\n"
        "### Описание задания:\n{instruction}\n\n### Входные данные:\n{input}\n\n### Ответ:"
    ),
    "prompt_no_input": (
        "Ниже приведено описание задания. "
        "Напиши ответ, который соответствует запросу.\n\n"
        "### Описание задания:\n{instruction}\n\n### Ответ:"
    ),
}


In [ ]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# превращение токенов в индексы и паддинг/урезание последовательностей

def _tokenize_fn(strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
    """Tokenize a list of strings."""
    tokenized_list = [
        tokenizer(
            text,
            return_tensors="pt",
            max_length=1024,
            truncation=True,
            padding=True,
        )
        for text in strings
    ]

    max_length = max(tokenized.input_ids.size(1) for tokenized in tokenized_list)

    input_ids = []
    labels = []
    for tokenized in tokenized_list:
        padded_input_ids = torch.cat([tokenized.input_ids[0], torch.zeros(max_length - tokenized.input_ids.size(1), dtype=torch.long)], dim=0)
        input_ids.append(padded_input_ids)
        labels.append(padded_input_ids)

    input_ids = torch.stack(input_ids, dim=0)
    labels = torch.stack(labels, dim=0)

    input_ids_lens = labels_lens = [max_length] * len(tokenized_list)

    return dict(
        input_ids=input_ids,
        labels=labels,
        input_ids_lens=input_ids_lens,
        labels_lens=labels_lens,
    )



In [ ]:
def preprocess(
    sources: Sequence[str],
    targets: Sequence[str],
    tokenizer: transformers.PreTrainedTokenizer,
) -> Dict:
    """Preprocess the data by tokenizing."""
    examples = [s + t for s, t in zip(sources, targets)]
    examples_tokenized, sources_tokenized = [_tokenize_fn(strings, tokenizer) for strings in (examples, sources)]
    input_ids = examples_tokenized["input_ids"]
    labels = copy.deepcopy(input_ids)
    for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
        label[:source_len] = IGNORE_INDEX
    return dict(input_ids=input_ids, labels=labels)

In [ ]:
# Далее это оборачивается к классы, которые предобрабатывают данные к формату huggingface.
class SupervisedDataset(Dataset):
    """Dataset for supervised fine-tuning."""

    def __init__(self, data_path: str, tokenizer: transformers.PreTrainedTokenizer):
        super(SupervisedDataset, self).__init__()
        logging.warning("Loading data...")
        list_data_dict = json.load(open(data_path))

        logging.warning("Formatting inputs...")
        prompt_input, prompt_no_input = PROMPT_DICT["prompt_input"], PROMPT_DICT["prompt_no_input"]
        sources = [
            prompt_input.format_map(example) if example.get("input", "") != "" else prompt_no_input.format_map(example)
            for example in list_data_dict
        ]
        targets = [f"{example['output']}{tokenizer.eos_token}" for example in list_data_dict]

        logging.warning("Tokenizing inputs... This may take some time...")
        data_dict = preprocess(sources, targets, tokenizer)

        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])


@dataclass
class DataCollatorForSupervisedDataset(object):
    """Collate examples for supervised fine-tuning."""

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=IGNORE_INDEX)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )

In [ ]:

json_data = []

for example in dataset['train']:
    json_example = {
        'instruction': example['instruction'],
        'input': example['input'],
        'output': example['output']
    }
    json_data.append(json_example)

with open('dataset.json', 'w', encoding='utf-8') as f:
    json.dump(json_data, f, ensure_ascii=False, indent=4)


In [ ]:
data_json = json.load(open('dataset.json'))

In [ ]:
print(tokenizer.pad_token_id)

128001


In [ ]:
data_json[:3]

[{'instruction': 'Опишите, как сделать горшок из глины.',
  'input': '',
  'output': 'Для изготовления горшка из глины сначала необходимо взять глину и размешать ее водой до состояния пластилина. Затем нужно сформировать глиняный комок нужной формы и тщательно откачать влагу из глины, чтобы она стала твердой. Горшок нужно обжечь в духовке при высокой температуре, чтобы он стал прочным и непроницаемым.'},
 {'instruction': 'Опиши процесс изготовления торта. Укажи ингредиенты, необходимые для этого.',
  'input': '',
  'output': 'Для изготовления торта необходимы следующие ингредиенты: мука, сахар, яйца, молоко, сливочное масло, разрыхлитель, ванилин. Сначала нужно смешать муку с сахаром и добавить яйца, молоко, разрыхлитель и ваниль. Затем тесто надо хорошо перемешать и разделить на несколько частей. Каждую часть нужно запекать в духовке до готовности. После этого торт надо собрать, сложив слои пирога вместе с начинкой и украшениями.'},
 {'instruction': 'Напиши аналог данной пословицы на 

## Fine-tuning

In [ ]:
for param in model.parameters():
  param.requires_grad = False
  if param.ndim == 1:
    # в layernorm нужны очень маленькие числа, поэтому для него оставляют fp32
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()
model.enable_input_require_grads()

В этой ячейке все веса изначальной модели замораживаются

In [ ]:
# вспомогательная функция которая покажет сколько параметров будут обучаться
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

Адаптеры можно добавлять к полносвязным/линейным/dense слоям. В зависимости от архитектуры эти слои могут называться по-разному, поэтому их нужно указать вручную через парметр `target_modules`. Так как модели это в основном наслоенные однотипные трансформерные блоки, то перечислить нужно лишь несколько имен типовых слоев.

In [ ]:
# вот так можно (в torch) напечатать слои и их названия
# в нашем случае на них прямо написано, что они линейные (proj и fc)
# поэтому мы можем выбрать q_proj, v_proj, k_proj, out_proj, fc1 и fc2
for name, module in model.named_modules():
    print(name)


model
model.embed_tokens
model.layers
model.layers.0
model.layers.0.self_attn
model.layers.0.self_attn.q_proj
model.layers.0.self_attn.k_proj
model.layers.0.self_attn.v_proj
model.layers.0.self_attn.o_proj
model.layers.0.self_attn.rotary_emb
model.layers.0.mlp
model.layers.0.mlp.gate_proj
model.layers.0.mlp.up_proj
model.layers.0.mlp.down_proj
model.layers.0.mlp.act_fn
model.layers.0.input_layernorm
model.layers.0.post_attention_layernorm
model.layers.1
model.layers.1.self_attn
model.layers.1.self_attn.q_proj
model.layers.1.self_attn.k_proj
model.layers.1.self_attn.v_proj
model.layers.1.self_attn.o_proj
model.layers.1.self_attn.rotary_emb
model.layers.1.mlp
model.layers.1.mlp.gate_proj
model.layers.1.mlp.up_proj
model.layers.1.mlp.down_proj
model.layers.1.mlp.act_fn
model.layers.1.input_layernorm
model.layers.1.post_attention_layernorm
model.layers.2
model.layers.2.self_attn
model.layers.2.self_attn.q_proj
model.layers.2.self_attn.k_proj
model.layers.2.self_attn.v_proj
model.layers.2.

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=32, # внутренняя размерность адаптера, основной параметр
    target_modules=["q_proj", "k_proj", "v_proj", 'out_proj', 'fc1', 'fc2'], # к каким слоям добавлять адаптеры (подробнее выше)

    # "вес" адаптера, этот параметр делится на r, то есть если они равны то
    # вес адаптера = 1 (то есть базовая модель и адаптер одинаковы по значимости)
    # если поставить этот параметр выше, то адаптер будет сильнее влиять на базовую модель
    # как я понимаю никто особо не понимает что делать с этим параметром при обучении
    # лучше оставлять его равным r
    lora_alpha=32,

    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 18874368 || all params: 8049135616 || trainable%: 0.23448937749889193


Параметр r отвечает за внутренюю размерность дополнительных матриц. При r=32 обучаться в итоге будет около 15м параметров, что меньше 1 процента от всех параметров изначальное модели.

In [ ]:
for attr, value in tokenizer.__dict__.items():
    print(f"{attr}: {value}")

_tokenizer: <tokenizers.Tokenizer object at 0x7fe5c06af630>
_decode_use_source_tokenizer: False
init_inputs: ()
init_kwargs: {'bos_token': AddedToken("<|begin_of_text|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True), 'clean_up_tokenization_spaces': True, 'eos_token': AddedToken("<|end_of_text|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True), 'model_input_names': ['input_ids', 'attention_mask'], 'model_max_length': 1000000000000000019884624838656, 'pad_token': AddedToken("<|end_of_text|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True), 'name_or_path': 'tokenizer-Llama3-ChatQA-1.5-8B', 'max_length': 1024, 'truncation_side': 'right', 'stride': 0, 'truncation_strategy': 'longest_first', 'pad_token_type_id': 0, 'padding_side': 'right', 'pad_to_multiple_of': None}
name_or_path: tokenizer-Llama3-ChatQA-1.5-8B
_processor_class: None
model_max_length: 1000000000000000019884624838656
padding_side

In [ ]:
train_dataset = SupervisedDataset(tokenizer=tokenizer, data_path='dataset.json')

In [ ]:

trainer = transformers.Trainer(
    model=model,
    train_dataset=train_dataset,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=100,
        max_steps=20,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir='outputs'
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False, )
)

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
torch.cuda.empty_cache()
trainer.train()

/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
1,0.670900
2,0.692300
3,0.782800
4,0.660000
5,0.645400
6,0.642400
7,0.567800
8,0.574000
9,0.525000
10,0.468300


Step,Training Loss
1,0.670900
2,0.692300
3,0.782800
4,0.660000
5,0.645400
6,0.642400
7,0.567800
8,0.574000
9,0.525000
10,0.468300


TrainOutput(global_step=20, training_loss=0.4859242826700211, metrics={'train_runtime': 1060.8762, 'train_samples_per_second': 0.151, 'train_steps_per_second': 0.019, 'total_flos': 7258961313792000.0, 'train_loss': 0.4859242826700211, 'epoch': 0.005365166655489236})

In [ ]:
model.save_pretrained('Llama3-ChatQA-1.5-8B_lora')

In [ ]:
!nvidia-smi

Thu May 16 16:19:50 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   76C    P0              41W /  70W |  14409MiB / 15360MiB |      3%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Сравнение ответов

Сохраним ~~модель~~ адаптер (сохранятся только дополнительные веса)

Чтобы загрузить обученный адаптер нужно сначала загрузить базовую модель, а потом применить к ней LoRa веса

In [ ]:
# перед запуском этой ячейки нужно перезапустить кернел
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

peft_model_id = "Llama3-ChatQA-1.5-8B_lora"

quantization_config = BitsAndBytesConfig(
        load_in_8bit=True
    )
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path="Llama3-ChatQA-1.5-8B",
                                             return_dict=True,
                                             quantization_config=quantization_config,
                                             device_map='auto'
                                            )

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:167: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("tokenizer-Llama3-ChatQA-1.5-8B")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
def generate(text, tokenizer, model):
    batch = tokenizer(text, return_tensors='pt').to('cuda')
    output_tokens = model.generate(**batch, max_new_tokens=50, temperature=0.1, do_sample=True, no_repeat_ngram_size=3)

    return tokenizer.decode(output_tokens[0], skip_special_tokens=True)

Давайте попробуем сделать несколько предсказаний, используя базовую модель, чтобы потом сравнить с дообученной

In [ ]:
print(generate("Расскажи анекдот про Штирлица",  tokenizer, model))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Расскажи анекдот про Штирлица» (вышел в 2003 году). В 2004 году вышел сборник «Всё о Штилце». В 2010 году вышла книга «Штирлициана. Сборник


In [ ]:
print(generate("Предскажи фильм, который выиграет Оскар в следующем году",  tokenizer, model))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Предскажи фильм, который выиграет Оскар в следующем году». В 2019 году он был номинирован на премию «Эмми» за «Выдающийся комментаторский вклад в программу» за свою работу на «Сегодня вечером».

В 2018 году


In [ ]:
print(generate("Придумай словесную игру",  tokenizer, model))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Придумай словесную игру с использованием слова "птица". Например: "Птица летает высоко"


In [ ]:
print(generate("Дай рецепт кваса",  tokenizer, model))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Дай рецепт кваса из дрожжей. Как сделать квас из дробжей в домашних условиях. Рецепт квасов из дровжей.

Квас из пивных дрожджей. Квас на дрож


In [ ]:
print(generate("Расскажи забавную шутку или каламбур",  tokenizer, model))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Расскажи забавную шутку или каламбур, связанный с темой "Домашнее животное"


Загрузим LoRA веса и попробуем те же промпты

In [ ]:
model = PeftModel.from_pretrained(model, peft_model_id)


In [ ]:
print(generate("Расскажи анекдот про Штирлица",  tokenizer, model))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Расскажи анекдот про Штирлица» (1989), «Почему Штиллица не любят в Германии?» (1990), «Как Штильц разговаривает со своей женой» (2000), а также «Какой


In [ ]:
print(generate("Предскажи фильм, который выиграет Оскар в следующем году",  tokenizer, model))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Предскажи фильм, который выиграет Оскар в следующем году».

В 2019 году «Лунную смену» назвали лучшим фильмом года на сайте Metacritic. В 2020 году «Манк» занял первое место в категории «Лучший филь


In [ ]:
print(generate("Придумай словесную игру",  tokenizer, model))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Придумай словесную игру, используя слова: «солнце», «лунный», «день», «ночь» и «звезда».


In [ ]:
print(generate("Дай рецепт кваса",  tokenizer, model))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Дай рецепт кваса на дрожжах. Дрожжи для квасов купить можно в любом магазине. Добавляете дрожжи в бродильный пакетик, заливаете водой и ставите в теплое место


In [ ]:
print(generate("Расскажи забавную шутку или каламбур",  tokenizer, model))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Расскажи забавную шутку или каламбур, связанный с темой "Сердце" или "Любовь":

- Сердце - это орган, который бьется от любви.
- Любовь - это чувство, которое бьет из сердца


In [ ]:
print(generate("Напиши стихотворение о природе",  tokenizer, model))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Напиши стихотворение о природе и ее красоте. Используй слова: «лес», «река», «гора», «небо», «цветы», «птицы», «звезды».
